Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import product
from random import random, randint, shuffle, seed,choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy


In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [20]:
PROBLEM_SIZE = 100  #aka NUM_POINTS because it represents the number of points to fully cover
NUM_SETS = 100
DENSITY = .3
x = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, DENSITY)
SETS = x.toarray()
print(SETS)


[[ True  True False ... False  True False]
 [ True False False ...  True False  True]
 [False False False ... False  True  True]
 ...
 [False False  True ... False False  True]
 [False False False ... False False False]
 [False False False ...  True False  True]]


In [67]:

def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost



In [64]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [71]:
def simulated_annealing(current_state, fitness, tweak, num_steps=1_000, init_temp=1.0, cool_factor=0.99):
    current_fitness = fitness(current_state)
    temp = init_temp
    counter = 0
    for step in range(num_steps):
        #print("Step ",step)
        new_state = tweak(current_state)
        new_fitness = fitness(new_state)
        
        #if the new solution is better, use it as current solution
        if new_fitness >= current_fitness:
            current_state, current_fitness = new_state, new_fitness
            #print("Current_fitness: ",new_fitness)
            counter+=1
        else:
            # otherwise, change the current solution with a probability bound to the actual temp but also linked to the new_fitness validity
            prob = np.exp((new_fitness[0]*new_fitness[1] - current_fitness[1]) / temp)
            if random() < prob:
                current_state, current_fitness = new_state, new_fitness
                #print("Current_fitness but because prob bound to temp: ",new_fitness)

        #the system is cooled down with a specific cool_factor
        temp *= cool_factor

    return current_state,counter




In [72]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#Use the simulated_annealing alg.
for i in range(10):
    current_state,fit_counter = simulated_annealing(current_state, fitness, tweak, num_steps=10000, init_temp=1.0 - i/100, cool_factor=0.99 - i/100)
    print(fitness(current_state),fit_counter)
    
    

(100, -7) 13
(100, -7) 0
(100, -7) 0
(100, -7) 0
(100, -7) 0
(100, -7) 0


C:\Users\Marco\AppData\Local\Temp\ipykernel_9252\978872247.py:17: RuntimeWarning: overflow encountered in divide
  prob = np.exp((new_fitness[0]*new_fitness[1] - current_fitness[1]) / temp)


(100, -7) 0
(100, -7) 0
(100, -7) 0
(100, -7) 0
